# Lesson 2 : State Farm

In [1]:
%pwd

u'/home/ubuntu/nbs'

In [2]:
import os

In [16]:
#Path of current directory and the data directory
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/statefarm/'

In [20]:
#setup the folder for the validation data
%mkdir -p $DATA_HOME_DIR'valid'

In [22]:
#create the classes for the validation data
for root, dirs, files in os.walk(DATA_HOME_DIR+'train/'):
    for name in dirs:
        os.mkdir(os.path.join(DATA_HOME_DIR+'valid', name))

In [23]:
import numpy as np
from glob import glob

In [38]:
#move 20% of the training data to the respective validation folders
for root, dirs, files in os.walk(DATA_HOME_DIR+'train/'):
    for name in dirs:
        g = glob(DATA_HOME_DIR + 'train/' + name + '/*.jpg')
        shuffle = np.random.permutation(g)
        for i in range(int(0.2*shuffle.size)): 
            os.rename(shuffle[i], DATA_HOME_DIR + 'valid/' + name + '/' + os.path.basename(shuffle[i]))

In [40]:
#setup the test data
%mkdir -p $DATA_HOME_DIR'test/none/'

In [44]:
#move all the files in the folder created above
%cd $DATA_HOME_DIR'test/'
%mv *.jpg  none/

/home/ubuntu/nbs/data/statefarm/test
